# Calculate reproduction rate (R0) for infectious diseases within a micro-environment
This model calculates the reproduction rate (R0) of infectious diseases (Covid19) within a micro-environment based upon dispersion of an aerosol within an enclosed space. It assumes the prime method of transmission is an aerosol and that the aerosol is well mixed across the space, i.e. there are no local concentrations which may impact on an individual's probability of being infected.

The modelling follows the approach set out in the paper:
Buonanno, G., Stabile, L., & Morawska, L. (2020). Estimation of airborne viral emission: Quanta emission rate of SARS-CoV-2 for infection risk assessment [Preprint]. Infectious Diseases (except HIV/AIDS). https://doi.org/10.1101/2020.04.12.20062828

In [1]:
import math
import plotly.graph_objects as go
import simpy
import pandas as pd

# Import local libraries
from Simulation import Simulation

In [2]:
periods = 100
simulation_name = 'Pharmacy'
simulation_run = 1

simulation = Simulation(simulation_name, simulation_run)
simulation.run(periods, report_time=True)

Running the model for 100 periods
Simulation finished. Execution time:0.015 seconds


In [3]:
result = simulation.get_results('Quanta concentration')
result

,simulation_name,simulation_run,time,quanta_concentration
0,Pharmacy,1,0,0.000000
1,Pharmacy,1,1,0.032667
2,Pharmacy,1,2,0.064157
3,Pharmacy,1,3,0.094514
4,Pharmacy,1,4,0.123778
...,...,...,...,...
95,Pharmacy,1,95,0.013841
96,Pharmacy,1,96,0.013343
97,Pharmacy,1,97,0.012862
98,Pharmacy,1,98,0.012399


In [4]:
time_axis = result['time'].tolist()
results = result['quanta_concentration'].tolist()

fig = go.Figure()
fig.add_trace(go.Scatter(x=time_axis, y=results , name='quanta concentration', line=dict(color='firebrick', width=1)))
fig.update_layout(title='Quanta concentration',
                   xaxis_title='Time (minutes)',
                   yaxis_title='Quanta concentration (quanta per m^-3)',
                   width=1280,
                   height=720)
fig.show()

In [5]:
# Visitor log
visitor_log = simulation.get_results('Visitor activity')
visitor_log

,simulation_name,simulation_run,time,queue,visitors,activity
0,Pharmacy,1,0.0,0,1,Visitor 0 requests entry.
1,Pharmacy,1,0.0,0,1,Visitor 0 entered.
2,Pharmacy,1,0.6,0,2,Visitor 1 requests entry.
3,Pharmacy,1,0.6,0,2,Visitor 1 entered.
4,Pharmacy,1,1.2,0,3,Visitor 2 requests entry.
...,...,...,...,...,...,...
257,Pharmacy,1,97.2,113,5,Visitor 162 requests entry.
258,Pharmacy,1,97.8,114,5,Visitor 163 requests entry.
259,Pharmacy,1,98.4,115,5,Visitor 164 requests entry.
260,Pharmacy,1,99.0,116,5,Visitor 165 requests entry.


In [6]:
result.to_excel('./output/results.xlsx', 'visitor_log')
visitor_log.to_excel('./output/visitor_log.xlsx', 'visitor_log')